In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando dados
train = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

In [ ]:
# Verificando tamanho dos dataframes
train.shape, test.shape

Podemos observar que o conjunto de dados de treino possui **103904** linhas e 25 colunas. O conjunto de dados de teste, por sua vez, possui **25976** linhas e 25 colunas.

In [ ]:
# Consolidando conjuntos de dados de treino e testes, para tratamento dos dados
df = train.append(test)
df.shape

Agora com um único dataframe, vamos explorar os dados e realizar os tratamentos necessários.

In [ ]:
# Visualizando algumas linhas do dataframe
df.head().T

In [ ]:
# Verificando o número de linhas do dataframe e quais os tipos dos dados
df.info()

In [ ]:
# Listando as colunas do dataframe que são do tipo object
df.select_dtypes("object").head()

Possuímos cinco colunas classificadas como object (variáveis categóricas), mas que pode ser transformadas em colunas numéricas. Para trabalhar com algoritmos de Machine Learning é importante fazer essa preparação dos dados. Outro ponto positivo dessa transformação nos dados é a redução do uso de memória.

In [ ]:
# Verificando missing values
df.isnull().sum()

Podemos observar que a coluna "Arrival Delay in Minutes", possui 393 valores classificados como nulos. Talvez a informação de fato não exista ou simplesmente não foi coletada. O problema de ausência de dados é muito comum, mas precisa ser tratado, visto que os modelos de Machine Learning não são capazes de lidar com valores ausentes.

Como alternativas de tratamento desse problema, podemos excluir esses dados ou preencher esses valores com a média ou com a moda, não existe receita de bolo para esse tratamento. Devemos escolher a abordagem que melhor se adeque ao nosso problema.

Vamos preencher os valores da faltantes com a moda da coluna "Arrival Delay in Minutes".

In [ ]:
# Preenchendo valores nulos com a moda
df["Arrival Delay in Minutes"].fillna(df["Arrival Delay in Minutes"].mode()[0], inplace=True)

In [ ]:
# Verificando novamente a existência de missing values
df.isnull().sum()

Podemos observar que a coluna "Arrival Delay in Minutes" já não possui mais valores nulos.

In [ ]:
# estatística descritiva e a frequência dos dados
df.describe().T

In [ ]:
# Excluindo colunas "Unnamed: 0" e "Id"
df.drop(["Unnamed: 0", "id"], axis=1, inplace=True)

In [ ]:
# Dataframe pós exclusão de colunas desnecessárias
df.info()

In [ ]:
# Criando cópia do dataframe
df2 = df.copy()

In [ ]:
# Verificando cópia do dataframe
df.shape, df2.shape

Agora vamos explorar os dados das colunas classificadas como object.

In [ ]:
# Coluna satisfaction

satisfaction = df['satisfaction'].unique()
count = df['satisfaction'].value_counts()

plt.bar(satisfaction, count, width=0.6, color='#4682B4')
plt.xlabel('Satisfaction')
plt.show()

In [ ]:
# Coluna Gender

gender = df['Gender'].unique()
count = df['Gender'].value_counts()

plt.bar(gender, count, width=0.6, color='#4682B4')
plt.xlabel('Gender')
plt.show()

In [ ]:
# Coluna Type of Travel

type_travel = df['Type of Travel'].unique()
count = df['Type of Travel'].value_counts()

plt.bar(type_travel, count, width=0.6, color='#4682B4')
plt.xlabel('Type of Travel')
plt.show()

In [ ]:
# Coluna Class

classe = df['Class'].unique()
count = df['Class'].value_counts()

plt.bar(classe, count, width=0.6, color='#4682B4')
plt.xlabel('Class')
plt.show()


Verificando a correlação entre as variáveis.

In [ ]:
# Correlação entre variáveis do dataframe

plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(),
            annot = True,
            fmt = '.2f',
            cmap='Blues')
plt.title('Correlação entre variáveis do dataframe de Satisfação de Usuários')
plt.show()

In [ ]:
# Criando variáveis dummy para variáveis categóricas do dataframe, definidas com o tipo "object"

df.select_dtypes("object").head()

In [ ]:
# Convertendo variável satisfaction

df['satisfaction'] = df['satisfaction'].replace({'satisfied': 1, 'neutral or dissatisfied': 0}).astype(int)

In [ ]:
# Verificando variável satisfaction

df['satisfaction'].value_counts()

In [ ]:
# Dummy para Gender

pd.get_dummies(df['Gender'])

In [ ]:
# Dummy para Customer Type

pd.get_dummies(df['Customer Type'])

In [ ]:
# Dummy para Type of Travel

pd.get_dummies(df['Type of Travel'])

In [ ]:
# Dummy para Class

pd.get_dummies(df['Class'])

In [ ]:
# Criando variáveis dummies para todas as colunas object

df = pd.get_dummies(df, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'])

In [ ]:
# Verificando tipos dos dados do dataframe após criação das variáveis dummies

df.info()

In [ ]:
# Verificando os dados após criação das dummies
df.head().T

In [ ]:
df.shape

**Criação do modelo**

In [ ]:
# Importando o train_test_split

from sklearn.model_selection import train_test_split

In [ ]:
# Separando dataframes treino e teste

train, test = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
# Separando dataframes treino e validação

train, valid = train_test_split(train, test_size=0.20, random_state=42)

In [ ]:
# Verificando dataframes separados

train.shape, valid.shape,test.shape

In [ ]:
# Definindo colunas de entrada

feats = [c for c in df.columns if c not in['satisfaction']]
feats

**Treinamento do modelo**

In [ ]:
# Importando modelo

from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instanciando o modelo

rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# Treinamento do modelo

rf.fit(train[feats], train['satisfaction'])

In [ ]:
# Prevendo dados de validação

preds_val = rf.predict(valid[feats])
preds_val

**Avaliando desempenho do modelo**

In [ ]:
# Importando a métrica

from sklearn.metrics import accuracy_score

In [ ]:
# Verificando a acurácia das previsões de validação

accuracy_score(valid['satisfaction'], preds_val)

In [ ]:
# Verificando a acurácia nos dados de teste

preds_test = rf.predict(test[feats])
accuracy_score(test['satisfaction'], preds_test)

In [ ]:
# Observando a coluna satisfaction do dataframe completo

df['satisfaction'].value_counts()

In [ ]:
df2.info()

In [ ]:
# Avaliando a de cada vaiável de entrada do modelo

fig=plt.figure(figsize=(10, 15))
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

**Gerando o gráfico de Matriz de Confusão**

In [ ]:
# Importando biblioteca

import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Teste

skplt.metrics.plot_confusion_matrix(test['satisfaction'], preds_test)

In [ ]:
# Matriz de confusão - Dados de Validação

skplt.metrics.plot_confusion_matrix(valid['satisfaction'], preds_val)